# Multi-class Classification

In this exercise is used the [Sign Language MNIST](https://www.kaggle.com/datamunge/sign-language-mnist) dataset, which contains 28x28 images of hands depicting the 26 letters of the english alphabet.


In [ ]:
import csv
import string
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img

In [ ]:
# Download the training and test sets:
!gdown --id 1z0DkA9BytlLxO1C0BAWzknLyQmZAp0HR
!gdown --id 1z1BIj4qmri59GWBG4ivMNFtpZ4AXIbzg

In [ ]:
# Define globals with the path to both files:

TRAINING_FILE = './sign_mnist_train.csv'
VALIDATION_FILE = './sign_mnist_test.csv'

# Each file includes a header (the first line) and 
# each subsequent data point is represented as a line that contains 785 values.

# The first value is the label (the numeric representation of each letter) and 
# the other 784 values are the value of each pixel of the image. 

In [ ]:
# This function reads a file passed as input and return 2 numpy arrays: 
# labels and 28x28 representation of each image

def parse_data_from_input(filename):
    with open(filename) as file:
        next(file)
        csv_data = np.loadtxt(file, delimiter = ',')
        labels = np.array(csv_data[:,0], dtype = np.float64)
        
        def reshape_element(element):
            return np.reshape(element, (28,28))

    images = np.array(list(map(reshape_element, csv_data[:,1:])), dtype = np.float64)

    return images, labels

In [1]:
# defining datagenerators for training and validation data

def train_val_generators(training_images, training_labels, validation_images, validation_labels):

    training_images = np.expand_dims(training_images, axis = -1)
    
    validation_images = np.expand_dims(validation_images, axis = -1)
    
    train_datagen = ImageDataGenerator(rescale = 1./255,
                                       fill_mode='nearest')
    
    train_generator = train_datagen.flow(x=training_images,
                                         y=training_labels,
                                         batch_size=32)
    
    validation_datagen = ImageDataGenerator(rescale = 1./255)
    
    validation_generator = validation_datagen.flow(x=validation_images,
                                                 y=validation_labels,
                                                 batch_size=32)

    return train_generator, validation_generator

In [ ]:
# defining the model itself

def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (28,28,1)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(512, activation = 'relu'),
        tf.keras.layers.Dense(26, activation = 'softmax')
    ])
    
    model.compile(optimizer = 'rmsprop',
                  loss = 'sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
model = create_model()

history = model.fit(train_generator,
                    epochs=15,
                    validation_data=validation_generator)